# These are the main notebook commands used in the presentation of julia at NU, Dec. 20th, 2016



# Basics

In [ ]:
versioninfo(true)

In [ ]:
Pkg.status()

In [ ]:
1 + 2

In [ ]:
1. + 2

In [ ]:
1/2  # floating point division

In [ ]:
2^2  # ^, not **

In [ ]:
1//2  # Rational type, not integer division

In [ ]:
1//2 + 3//4

In [ ]:
x = 0
sin(x)
θ = x
5sin(2θ)

In [ ]:
conj(1 + 3*im)

In [ ]:
randn(5,5)

In [ ]:
1 + 3 * im

In [ ]:
conj(1 + 3*im)

In [ ]:
(1 + 3im) * (2 + 4im)

In [ ]:
println("Hello, world!")  # double quotes only
"This is a string!"  # double quotes only

Complex numbers and arbitrary-precision arithmetic (via MPFR) are available, of course.

In [ ]:
cos(big(3 + 4im))

In [ ]:
factorial(big(10000))

# Plots

In [ ]:
using PyCall, PyPlot # Mandelbrot set requires V 0.3.12

In [ ]:
t = linspace(0,2pi,201)
plot(t, sin(t), linewidth=2)

In [ ]:
using PyPlot

col = linspace(0,2pi,250)
t = [1 2 3 4 5] .* col

plot( sin(t), linewidth = 2 )

In [ ]:
plot( sin(col) + rand(250) )

In [ ]:
plot(exp(sin(1:0.1:100)))

# Mandelbrot Set

In [ ]:
# julia set
# (the familiar mandelbrot set is obtained by setting c==z initially)
function julia(z, c; maxiter=200)
    for n = 1:maxiter
        if abs2(z) > 4
            return n-1
        end
        z = z*z + c
    end
    return maxiter
end

### Use version 0.3.12 of julia for the following command to work!

In [ ]:
@time m = [ uint8(julia(complex(r,i), complex(-.06,.65))) for i=1:-.002:-1, r=-1.5:.002:1.5 ];

# display a color map   http://matplotlib.org/api/cm_api.html
get_cmap("RdGy")

In [ ]:
imshow(m, cmap="RdGy", extent=[-1.5,1.5,-1,1])

# Arrays

In [ ]:
v = [1, 2, 3, 4]  # array literal syntax

In [ ]:
vv = [1 2 3 4]  # horizontal concatenation (hcat)

In [ ]:
vvv = [1 ; 2 ; 3 ; 4]  # vertical concatenation (vcat)

In [ ]:
v[1]  # indices start at 1, no negative indices (at least not for indexing from the end)

In [ ]:
v[2:end]  # end keyword --> length(v)  

In [ ]:
A = [[1 2 3] ; [4 5 6]]

In [ ]:
A'  # transpose

In [ ]:
A[:]

In [ ]:
A = rand(10,300)

In [ ]:
b = rand(10) # a random rank-10 vector
x = A \ b    # solve for x satisfying A*x = b; has rank 300
B = A' * A   # A-transpose multiplied by A; is a 300x300 symmetrix matrix with real eigenvalues
eigvals(B)

# Unicode support

In [ ]:
x = 20
y = x + 20
θ = y + 10
println("θ = $θ")

In [ ]:
√81

In [ ]:
MYMAX = 2048
x = rand(MYMAX)
μ = .25
σ = .45

# Comprehensions, etc.

In [ ]:
[i for i in 1:10]

In [ ]:
x = [5i for i in 0:20 if i%2 == 0]

In [ ]:
x[20]

In [ ]:
for i = 1:5
    println("Hello, World")
end 

# Defining functions
 Julia function definitions syntax is pretty standard:

In [ ]:
f(x) =  x^2 + x -3
f(2.3)
# function doubler(x)
#     return 2x  # okay to have literal * variable without *
# end

In [ ]:
doubler(2), doubler(6)

In [ ]:
firstelem = x -> x[1]

In [ ]:
firstelem("tester")

# Defaults and keyword arguments
 Julia supports two common use cases: defaults and keyword arguments. These are a little less flexible than in Python but allow for effective compilation

In [ ]:
function Lpnorm(x, p=2)
    return sum(x.^p)^(1/p)
end

In [ ]:
aa = rand(10)
norm(aa), Lpnorm(aa), Lpnorm(aa, 1)

# Serial vs. parallel Buffon

In [ ]:
function buffon(m)
  hit = 0
  for l = 1:m
    mp  = rand()
    phi = (rand()* pi) - pi/2
    xrechts = mp + cos(phi)/2
    xlinks  = mp - cos(phi)/2
    if xrechts >= 1 || xlinks <= 0
        hit += 1
    end
    end
    miss = m - hit
  piapprox = m / hit * 2
end

In [ ]:
buffon(10^6)

In [ ]:
nprocs()==1 && addprocs(CPU_CORES - 1)

In [ ]:
function buffon_par(m)
  hit = @parallel (+) for l = 1:m
    mp = rand()
    phi = (rand() * pi) - pi / 2
    xrechts = mp + cos(phi)/2
    xlinks  = mp - cos(phi)/2
    (xrechts>=1 || xlinks<=0)? 1:0
  end
  miss = m - hit
  piapprox = m / hit * 2
end

In [ ]:
buffon(1000)      # warmup
buffon_par(1000)  # warmup
[@time buffon(10^8) for i in 1:5]
[@time buffon_par(10^8)  for i in 1:5];

In [ ]:
4.6/.46  # 10 times faster in parallel

# Type stability

In [ ]:
function sum1(N::Int)
    total = 0
    
    for i in 1:N
        total += i/2
    end
    
    total
end

function sum2(N::Int)
    total = 0.0
    
    for i in 1:N
        total += i/2
    end
    
    total
end

In [ ]:
sum1(10), sum2(10)

In [ ]:
N = 10000000
@time sum1(N)
@time sum2(N)

In [ ]:
code_llvm(sum1, (Int, ))

In [ ]:
code_native(sum1, (Int,))